### Defining the board name.

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'

### Configuration the CWNANO board and its ADC.

In [2]:
%run "../../../Helper_Scripts/Setup_Generic.ipynb"
scope.adc.samples = 1000
scope._cwusb.usbtx.rep = 0x81
scope._cwusb.usbtx.wep = 0x02

INFO: Found ChipWhisperer😍


### Programming board with the provided HEX file.

In [3]:
fw_path = "../search-CWNANO.hex"
cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13547 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13547 bytes


### Here we call search function for all nuber between 0-255. Also, we save the extracted power reaces in two format (.txt, .npy) as it is required by organizer.

In [4]:
import numpy as np
import time
from scipy.stats import pearsonr

power=np.zeros((256,1000), dtype=float)
for i in range(0,256):
    msg = bytearray([i]*1)
    scope.arm()
    target.simpleserial_write('s',msg)
    scope.capture()
    power[i][0:1000]=scope.get_last_trace()[0:1000]

np.save('search_power_file', power)
file = open("search_power_file.txt","w")
for i in range(256):
    for j in range(1000):
        file.write(str(power[i][j])+' ')
    file.write("\n")        
file.close()             

### Here you can load my power traces that I already stored. Please do not run this code if you want to extract power traces. My_search_power_file.npy and My_search_power_file.txt is my power traces when I ran code.

In [1]:
import numpy as np
import time
from scipy.stats import pearsonr
power=np.zeros((256,1000), dtype=float)
power = np.load('My_search_power_file.npy')

#f=open("My_search_power_file.txt","r")
#val = f.readlines()
#for i in range(256):
#    power[i] = val[i].strip().split()

In [6]:
#%run "../../../Helper_Scripts/plot.ipynb"
#plot = real_time_plot(plot_len=500)

### As we know in the search algorithim, 6 numbers between 0-255 are selected randomly and moved to the end of array. When we call the search function with these 6 numbers, the run time is longer than the other numbers that are not moved. 
### Based on this point we can analyze the power traces for each number, when the power traces has more samples, it means that i has longer time process. Therefore we can find which number are moved from array to the end of array.
### Here we analyze the time process of all numbers with their power traces, and then sort it from lowest time to highest time process. The last 6 numbers are the answer of this challenge. 

![search.png](search.png)

### For example, we show the poser traces for 6 cases when we called search function. Number: 91, 118, 207, 16, 123, 200. As you can see for the figures a, b, c the time process is at around 310, while for the second row the time process is lower than 310. Please note that, we used power traces as time process, as the only realtime component that we have access is the ADC of board. 
### Numbers 91, 118, 207 are our answers as they have longest time process. We analized all power traces for all number to find the first 6 numbers that have longest time to process.


In [7]:
time_list=[0]*256
for i in range(0,256):
    #plot.send(power[i][0:500]) 
    threshold=0#sum(signal[0:10])/10
    filtered_signal=[0]*500
    for j in range (0,500):
        if  (power[i][j] > threshold):
            filtered_signal[j]=1
        else:
            filtered_signal[j]=0  
    #print(filtered_signal[0:500])
    filtered_signal_str=str(filtered_signal).split('0, 0, 0, 0, 0, 0, 0')[0]
    ex_time=len(filtered_signal_str[1:])/3
    time_list[i] = ex_time

sort_index = np.argsort(time_list)
#print(time_list)

### After finding the 6 numbers, we need to check it with verify function. Here we call verify function.

In [8]:
msg = bytearray([0]*6)
msg[0] = sort_index[250]
msg[1] = sort_index[251]
msg[2] = sort_index[252]
msg[3] = sort_index[253]
msg[4] = sort_index[254]
msg[5] = sort_index[255]
target.simpleserial_write('a',bytearray(msg))
done=target.simpleserial_read('r', 1)
print("The 6 moved numbers: {}".format(list(msg)))
print("Response of Verify Function: {}".format(list(done)))

The 6 moved numbers: [207, 232, 152, 91, 59, 118]
Response of Verify Function: [1]


### This is the Answer:
### The 6 moved numbers : 
### [207, 232, 152, 91, 59, 118]